In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, TimeDistributed, LSTM, InputLayer
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.layers.normalization import BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

model = Sequential()
model.add(InputLayer(input_shape = (13, 300, 300, 3)))
model.add(TimeDistributed(Conv2D(32, (3, 3), activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model.add(TimeDistributed(Conv2D(32, (3, 3), activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3), activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

model.add(TimeDistributed(Flatten()))  # this converts our 3D feature maps to 1D feature vectors
model.add(LSTM(256, return_sequences = True))
model.add(TimeDistributed(Dense(64, activation = 'relu')))
model.add(TimeDistributed(Dense(1, activation = 'softmax')))

model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

model.summary()

Using TensorFlow backend.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 13, 298, 298, 32)  896       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 13, 149, 149, 32)  0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 13, 147, 147, 32)  9248      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 13, 73, 73, 32)    0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 13, 71, 71, 64)    18496     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 13, 35, 35, 64)    0         
____________________

In [ ]:
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray
import numpy as np
import os

train_path = '/content/drive/My Drive/Project/Data/RNNData/Train'
test_path = '/content/drive/My Drive/Project/Data/RNNData/Test'
#dataframe = pd.read_csv('Train_1.csv')

target_labels = pd.read_csv('/content/drive/My Drive/Project/Data/target_label.csv')

path = '/content/drive/My Drive/Project/Data/RNNData'

time_step = 13

train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

def OneHotEncodding(X, length):
    output = np.ndarray((len(X),13), dtype = np.int16)
    for i, value in enumerate(X):
        for y in range(1,14):
            if value == y:
                output[i][y - 1] = 1
            else:
                output[i][y - 1] = 0
    return output

#def image_gen(img_paths, img_size=(300, 300, 3), time_step = 13):
 #   global dataframe
  #  for img_path in img_paths:
   #     for i in range(time_step):
    #        # Load the image and mask, and normalize it to 0-1 range
     #       img = imread(train_path +  '/' +img_path) / 255.
      #      # Resize the images
       #     img = resize(img, img_size, preserve_range=True)
        #    dataset[i] = img
        #yield dataset
#x = image_gen(train_files)
#d = next(x)
#print(x)

#y = OneHotEncodding([12,4], 13)
encoded_target_labels = OneHotEncodding(target_labels['labels'], 13)
encoded_train_labels = encoded_target_labels[0:46]
encoded_test_labels = encoded_target_labels[46:]
#print(len(y))
#y = np.array(y)
#print(y.shape)
train_files.sort()
test_files.sort()

def image_gen(img_paths, name, img_size=(300, 300, 3), time_step = 13):
    global dataframe

    dataset = np.ndarray(shape=(int(len(img_paths)), 300, 300, 3),
                     dtype=np.float32)

    i = 0
    index = -1

    if name == 'Train':
      y = encoded_train_labels
    elif name == 'Test':
      y = encoded_test_labels
    for img_path in img_paths:
        #print(img_path)
        if i % time_step == 0:
            #print(y_label, y_label.shape)
            if index != -1:
                yield dataset[i-13:i], y_label
            index += 1
            if name == 'Train':
              y_label = y[int(i / 13) % 46]
            elif name == 'Test':
              y_label = y[int(i / 13) % 4]
            print(len(y))
            #y = dataframe.iloc[:,i:i+12]
            #y = OneHotEncodding([12], 13)
            #y = np.array(y)


        img = imread(path + '/' + name + '/' +img_path) / 255.
        img = resize(img, img_size, preserve_range=True)
        dataset[i] = img
        i += 1
    yield dataset[i-13:i], y_label
        #yield dataset, y

#x = image_gen(train_files, 'Train')
#print(type(x))
#d, t = next(x)
#print(d.shape, t.shape)

#y = image_gen(test_files, 'Test')
#d, t = next(y)
#print(d.shape, t.shape)

#d, t = next(x)
#print(d.shape, t.shape)



In [ ]:
def image_batch_generator(img_paths, name, batch_size= 32):
    while True:
        ig = image_gen(img_paths, name)
        batch_img, y_label = [], []

        for x , y in ig:
            # Add the image and mask to the batch
            batch_img.append(x)
            y_label.append(y)

            #print(y_label, len(y_label))
            # If we've reached our batchsize, yield the batch and reset
            #print(len(batch_img))
            if len(batch_img) == batch_size:
                yield np.stack(batch_img, axis=0), np.reshape(np.stack(y_label, axis = 0),(batch_size, time_step, 1))
                batch_img, y_label = [], []

        # If we have an nonempty batch left, yield it out and reset
        if len(batch_img) != 0:
            yield np.stack(batch_img, axis=0), np.stack(y_label, axis = 0)
            batch_img, y_label = [], []

train_generator = image_batch_generator(train_files,'Train', batch_size=1)
validation_generator = image_batch_generator(test_files,'Test', batch_size=1)

#x_data, y_label = next(train_generator)
#print(x_data.shape, y_label.shape)

#x_data, y_label = next(train_generator)
#print(x_data.shape, y_label.shape)

In [ ]:
#/content/drive/My Drive/Project/Models/RNN_Models/
save_callback = ModelCheckpoint('RNN_Models_RTRNN_save_best_epoch.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min', period=10)

class CustomModelCheckpoint(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # logs is a dictionary
        if epoch % 10 == 0:
          #print(f"epoch: {epoch}, train_acc: {logs['acc']}, valid_acc: {logs['val_acc']}")
          #if logs['val_acc'] > logs['acc']: # your custom condition
          self.model.save('RNN_Models_RTRNN' + str(epoch) + '_model.h5')

save_per_epoch = CustomModelCheckpoint()

history = model.fit_generator(
        train_generator,
        steps_per_epoch=46,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=4,
        callbacks = [save_callback, save_per_epoch])

Epoch 1/50
464

46
4
 2/46 [>.............................] - ETA: 1:03 - loss: 14.7160 - acc: 0.07694
46
 3/46 [>.............................] - ETA: 1:01 - loss: 14.7160 - acc: 0.07694
46
 4/46 [=>............................] - ETA: 57s - loss: 14.7160 - acc: 0.0769 4
46
 6/46 [==>...........................] - ETA: 49s - loss: 14.7160 - acc: 0.07694
46
 7/46 [===>..........................] - ETA: 47s - loss: 14.7160 - acc: 0.07694
46
 8/46 [====>.........................] - ETA: 46s - loss: 14.7160 - acc: 0.076946
4
10/46 [=====>........................] - ETA: 43s - loss: 14.7160 - acc: 0.07694
46
11/46 [======>.......................] - ETA: 42s - loss: 14.7160 - acc: 0.07694
4
46
45/46 [============================>.] - ETA: 1s - loss: 14.7160 - acc: 0.076946
46
4
46/46 [==============================] - 47s 1s/step - loss: 14.7160 - acc: 0.0769 - val_loss: 14.7160 - val_acc: 0.0769
Epoch 2/50
 2/46 [>.............................] - ETA: 24s - loss: 14.7160 - acc: 0.07694
46
